In [1]:
import numpy as np
from numpy import log as ln
import pandas as pd
import csv
from scipy.stats import linregress

with open('waveform_raw053123.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    row_count = sum(1 for row in reader)

rk_list = list(range(3, row_count))
for rk in rk_list:
    ##

    #Bring in raw data
    #Make sure the the mini Trace TDR csv file is converted to UTF-8 Encoding
    #select rows to keep, alway keep row 2, second row is the select waveform

    #rk = 24 #row to keep

    rows_to_keep = [2,rk]
    wf_rawB = pd.read_csv('waveform_raw053123.csv', skiprows= lambda x: x not in rows_to_keep).T
    header_row = 0
    wf_rawB.columns = wf_rawB.iloc[header_row]

    #Create name of sample
    ch_sample = str(wf_rawB.columns.values.tolist())



    rows_to_keep = [2,rk]
    wf_rawA = pd.read_csv('waveform_raw053123.csv', skiprows= lambda x: x not in rows_to_keep).T
    header_row = 0
    wf_rawA.columns = wf_rawB.iloc[header_row]

    wf_rawA.reset_index(drop=True, inplace=True)
    wf_rawA.index = np.arange(1,len(wf_rawA)+1)
    wf_rawA.reset_index(inplace=True)
    wf_rawA.set_axis(['Time', 'Sam'],axis=1,inplace=True)

    #Get Variables variables from imput data 
    #Pull out sample number and info


    ###ENTER Start and Finish Time if times need adjusting

    Ref_EC = 99997
    #Start Time
    t_start = wf_rawA[wf_rawA['Time'] == 8]['Sam']
    #Finish Time
    t_finish = wf_rawA[wf_rawA['Time'] == 9]['Sam']

    ##### Hash/UnHash if TRACE times are OK
    t_s = int(t_start) #un-hash to use Trace start time
    t_f = int(t_finish) #un hash if you don't have to manually change tf

    Probe_L = float(wf_rawA[wf_rawA['Time'] == 14]['Sam'])
    L = Probe_L/100

    #for graphing
    int_time = int(wf_rawA[wf_rawA['Time'] == 11]['Sam'])
    t_sg = (t_s /int_time)
    t_fg = (t_f /int_time)
    
   
      
     # Value sfrom Trace

    ch_sample = (str(ch_sample)).strip()
    Ka_trace = float(wf_rawA[wf_rawA['Time'] == 6]['Sam'])
    sol_number = (str((wf_rawA[wf_rawA['Time'] == 28]['Sam']))).strip(' Name: am, dtype: object\n')

    Date = str((wf_rawA[wf_rawA['Time'] == 3]['Sam'])).strip('\nName: Sam, dtype: object' )
    Time = str((wf_rawA[wf_rawA['Time'] == 4]['Sam'])).strip('\nName: Sam, dtype: object')
    Volumetric = float(wf_rawA[wf_rawA['Time'] == 5]['Sam'])
    Zero_Time = int(wf_rawA[wf_rawA['Time'] == 7]['Sam'])
    Probee_index = int(wf_rawA[wf_rawA['Time'] == 12]['Sam'])
    Probe_type = str((wf_rawA[wf_rawA['Time'] == 13]['Sam'])).strip('12    \nName: Sam, dtype: object')
    Probe_L = float(wf_rawA[wf_rawA['Time'] == 14]['Sam'])

    #Time of Flight
    t_flight = wf_rawA[wf_rawA['Time'] == 10]['Sam']


    #get Waveform data
    wf_rawB.reset_index(drop=True, inplace=True)
    wf = wf_rawB.drop(labels=range(0, 29), axis=0)

    wf.index = np.arange(1,len(wf)+1)
    wf.reset_index(inplace=True)
    wf.set_axis(['Time', 'Sam'],axis=1,inplace=True)


    #Make Graphs

    # percent moisture from Topp

    #Start Time in seconds
    tsa = t_s/1000000000000
    #Finish Time in Seconds
    tfa = t_f/1000000000000
    t_dif = tfa - tsa

    #Speed of light in vacuum m/s
    c = 299792458
    #Length of waveguide in meters


    #Apparent Permetivity for wet sand
    Ka = ((t_dif * c)/L)**2
    topp_wfv = (-530 + 292*Ka - 5.5*Ka**2 + .043*Ka**3)/100

    #Calculate the Voltages at the Key times

    v0 = int(wf[wf['Time'] == int(10)]['Sam'])


    v1a = min(wf['Sam'])


    v1 = int(wf[wf['Time'] == int(t_sg)]['Sam'])


    v3 = int(wf[wf['Time'] == int(t_fg)]['Sam'])


    #Slope after the final time
   
    rolltime_frame = 20

    def get_slope(array):
        y = np.array(array)
        x = np.arange(len(y))
        slope, intercept, r_value, p_value, std_err  = linregress(x,y)
        return slope


    wf['rolling_slope'] = wf['Sam'].rolling(window=rolltime_frame, center=True,
                               min_periods=rolltime_frame).apply(get_slope)

  
    sf = wf.loc[wf['Time'] > t_fg, 'rolling_slope'].max()
     
    #vf and vr parameters  
    
    
    vf = int(wf[wf['Time'] >t_fg]['Sam'].max())
    vr = vf - v3
    
    #Dalton 1986
    #Make EC Dalton 0 if it is negative
    def ec_dalton(Ka, L, v3, vr):
        EC_Daltona = ((Ka**0.5)/(120*3.14*L))*ln(v3/vr)
        if EC_Daltona < 0:
            EC_Daltona = 0
        return EC_Daltona

    EC_Dalton = ec_dalton(Ka, L, v3, vr) 


    #Topp 1988
    #Make EC Topp 0 if negative
    def ec_topp(Ka, L, v3, vr, v0):
        EC_Toppa = ((Ka**0.5)/(120*3.14*L))*ln((v3*(2*v0-v3))/(v0*vr))
        if EC_Toppa < 0:
            EC_Toppa = 0
        return EC_Toppa

    ECtopp = ec_topp(Ka, L, v3, vr, v0) 

    #Make Export DF

    report_df = pd.DataFrame({'ch_sample':[ch_sample],
                              'sol_number':[sol_number],
                              'Ref_EC':[Ref_EC],
                              'Date':[Date],
                              #'Time':[Time],
                              'Volumetric':[Volumetric],
                              'Ka_trace':[Ka_trace],
                              'Zero_Time':[Zero_Time],
                              't_s':[t_s],
                              't_f':[t_f],
                              't_dif':[t_dif],
                              'int_time':[int_time],
                              'Probee_index':[Probee_index],
                              'Probe_type':[Probe_type],
                              'Probe_L':[Probe_L],
                              'L':[L],
                              'Ka':[Ka], 
                              'v0':[v0], 
                              'v1a':[v1a],
                              'v1':[v1],
                              'sf':[sf], 
                              'v3':[v3],
                              'vf':[vf],
                              'vr':[vr],
                              'EC_Dalton':[EC_Dalton],
                              'ECtopp':[ECtopp]})


    #Send Data to CSV
    # read the existing CSV file as a DataFrame
   
    Report3a = pd.read_csv('model_input_B.csv', header=0)

    # append the new data to the existing DataFrame
    
    Report_4 = pd.concat([Report3a,report_df])

    # write the merged DataFrame to the existing CSV file
    
    Report_4.to_csv('model_input_B.csv', index=False)
    
    
    
    
    
    
    
    
    
    
    
    #print(ch_sample,)
    
    print(report_df)

  ch_sample            sol_number  Ref_EC          Date  Volumetric  Ka_trace  \
0    [2535]  27    Sol 7\nName: S   99997  2    4/11/23        -1.0      81.2   

   Zero_Time   t_s    t_f         t_dif  ...         Ka    v0   v1a    v1  \
0      20105  4100  10161  6.061000e-09  ...  82.541049  3787  2131  3590   

         sf    v3    vf   vr  EC_Dalton   ECtopp  
0  1.053383  2135  2359  224   0.271806  0.31545  

[1 rows x 25 columns]
  ch_sample            sol_number  Ref_EC          Date  Volumetric  Ka_trace  \
0    [2534]  27    Vat 2\nName: S   99997  2    4/11/23        -1.0      81.2   

   Zero_Time   t_s    t_f         t_dif  ...         Ka    v0   v1a    v1  \
0      20105  4100  10233  6.133000e-09  ...  84.513744  3787  2135  3587   

         sf    v3    vf   vr  EC_Dalton    ECtopp  
0  1.068421  2139  2363  224   0.275263  0.319336  

[1 rows x 25 columns]
  ch_sample            sol_number  Ref_EC          Date  Volumetric  Ka_trace  \
0    [2531]  27    Sol 2\nName: